In [18]:
# data reading and const setting
import pandas as pd
import numpy as np
import random
from enum import Enum
import math

In [19]:
class CrossoverType(Enum):
    PartialMappedCrossover = 1
    OrderCrossover = 2
    PositionBasedCrossover = 3
    
class MutationType(Enum):
    Inversion = 1
    Insertion = 2
    Displacement = 3
    ReciprocalExchange = 4
    
class SelectionType(Enum):
    Deterministic = 1
    Stochastic= 2
    
class GeneticAlgorithm:
    def __init__(self,pop_size,number_of_genes,selection_type,
                 crossover_type,crossover_rate,mutation_type,mutation_rate,
                 compute_objective_value):
        
        self.pop_size = pop_size
        self.selection_type = selection_type
        self.crossover_size = int(pop_size*crossover_rate)
        if(self.crossover_size%2==1):
            self.crossover_size -= 1;
        self.mutation_size =  int(pop_size*mutation_rate)
        self.total_size = self.pop_size+self.mutation_size+self.crossover_size
        self.number_of_genes = number_of_genes
        self.crossover_type = crossover_type
        self.crossover_rate = crossover_rate
        self.mutation_type = mutation_type
        self.mutation_rate = mutation_rate
        self.compute_objective_value = compute_objective_value
        self.least_fitness_factor = 0.3
        self.mapping = [-1 for i in range(self.number_of_genes)] #for crossover
        
    def initialize(self):
        self.selected_chromosomes = np.zeros((self.pop_size,self.number_of_genes))
        self.indexs = np.arange(self.total_size)
        self.chromosomes = np.zeros((self.total_size,self.number_of_genes),dtype=int)
#         self.chromosomes = np.array([[range(1, self.number_of_genes + 1)] * self.total_size], dtype=int)
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):  
                self.chromosomes[i][j] = j
            np.random.shuffle(self.chromosomes[i])
       
        for i in range(self.pop_size,self.total_size):
            for j in range(self.number_of_genes):
#                 self.chromosomes[i][j] = -1
                self.chromosomes[i][j] = j
                
        self.fitness = np.zeros(self.total_size) 
        self.objective_values = np.zeros(self.total_size)
        self.best_chromosome = np.zeros(self.number_of_genes,dtype=int)
        self.best_fitness = 0
        
    def evaluate_fitness(self):
        for i,chromosome in enumerate(self.chromosomes[:self.pop_size]):
            self.objective_values[i] = self.compute_objective_value(chromosome)
           
        min_obj_val = np.min(self.objective_values)
        max_obj_val = np.max(self.objective_values)
        range_obj_val = max_obj_val-min_obj_val
        
        for i,obj in enumerate(self.objective_values):
            self.fitness[i] = max(self.least_fitness_factor*range_obj_val,pow(10,-5))+\
                (max_obj_val-obj)
                
    def update_best_solution(self):
        best_index = np.argmax(self.fitness)
        if(self.best_fitness<self.fitness[best_index]):
            self.best_fitness = self.fitness[best_index]
            for i,gene in enumerate(self.chromosomes[best_index]):
                self.best_chromosome[i] = gene
    
    def shuffle_index(self,length):
        for i in range(length):
            self.indexs[i] = i
        np.random.shuffle(self.indexs[:length])
            
    def perform_crossover_operation(self):
        self.shuffle_index(self.pop_size)
        
        child1_index = self.pop_size
        child2_index = self.pop_size+1
        count_of_crossover = int(self.crossover_size/2)
        for i in range(count_of_crossover):
            parent1_index = self.indexs[i]
            parent2_index  = self.indexs[i+1]
            
            if(self.crossover_type == CrossoverType.PartialMappedCrossover):
                self.partial_mapped_crossover(parent1_index,parent2_index,child1_index,child2_index)
                self.objective_values[child1_index] = self.compute_objective_value(self.chromosomes[child1_index])
                self.objective_values[child2_index] = self.compute_objective_value(self.chromosomes[child2_index])
            
            child1_index +=2
            child2_index +=2
        
    def partial_mapped_crossover(self,p1,p2,c1,c2):
        #reset
        for i in range(self.number_of_genes):
            self.mapping[i] = -1
         
        rand1 = random.randint(0,self.number_of_genes-2)
        rand2 = random.randint(rand1+1,self.number_of_genes-1)
       
        for i in range(rand1,rand2+1):
            c1_gene = self.chromosomes[p2][i] 
            c2_gene = self.chromosomes[p1][i]
            
            if(c1_gene==c2_gene):
                continue
            
            elif(self.mapping[c1_gene]==-1 and self.mapping[c2_gene]==-1):
                self.mapping[c1_gene] = c2_gene
                self.mapping[c2_gene] = c1_gene
                
            elif(self.mapping[c1_gene]==-1):
                self.mapping[c1_gene] =  self.mapping[c2_gene]
                self.mapping[self.mapping[c2_gene]] = c1_gene
                self.mapping[c2_gene] = -2
                
            elif (self.mapping[c2_gene]==-1):
                self.mapping[c2_gene] =  self.mapping[c1_gene]
                self.mapping[self.mapping[c1_gene]] = c2_gene
                self.mapping[c1_gene] = -2
                
            else:
                self.mapping[self.mapping[c1_gene]] = self.mapping[c2_gene]
                self.mapping[self.mapping[c2_gene]] = self.mapping[c1_gene]
                self.mapping[c1_gene] = -3
                self.mapping[c2_gene] = -3
                
        for i in range(self.number_of_genes):
            if(i>=rand1 and i<=rand2):
                self.chromosomes[c1][i] =  self.chromosomes[p2][i]
                self.chromosomes[c2][i] =  self.chromosomes[p1][i]
            else:
                if(self.mapping[self.chromosomes[p1][i]] >=0):
                    self.chromosomes[c1][i] = self.mapping[self.chromosomes[p1][i]]
                else:
                    self.chromosomes[c1][i] =self.chromosomes[p1][i]        
                
                if(self.mapping[self.chromosomes[p2][i]] >=0):
                    self.chromosomes[c2][i] = self.mapping[self.chromosomes[p2][i]]
                else:
                    self.chromosomes[c2][i] =self.chromosomes[p2][i]
        
    def do_roulette_wheel_selection(self,fitness_list):
        sum_fitness = sum(fitness_list)
        transition_probability = [fitness/sum_fitness for fitness in fitness_list]
        
        rand = random.random()
        sum_prob = 0
        for i,prob in enumerate(transition_probability):
            sum_prob += prob
            if(sum_prob>=rand):
               return i
        
    def perform_selection(self):
        if self.selection_type == SelectionType.Deterministic:
            index = np.argsort(self.fitness)[::-1]
        
        elif self.selection_type == SelectionType.Stochastic:
            index = [self.do_roulette_wheel_selection(self.fitness) for i in range(self.pop_size)]
        
        else:
            index = self.shuffle_index(self.total_size)
        
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):
                self.selected_chromosomes[i][j] =  self.chromosomes[index[i]][j]
        
        for i in range(self.pop_size):
            for j in range(self.number_of_genes):
                self.chromosomes[i][j] = self.selected_chromosomes[i][j]
                
    def perform_mutation_operation(self):
        self.shuffle_index(self.pop_size+self.crossover_size)
        child1_index = self.pop_size+self.crossover_size
        for i in range(self.mutation_size):
            if(self.mutation_type==MutationType.Inversion):
                parent1_index = self.indexs[i]
                self.inversion_mutation(parent1_index,child1_index)
                self.objective_values[child1_index] = self.compute_objective_value(self.chromosomes[child1_index])
                child1_index += 1
            
    def inversion_mutation(self,p1,c1):
        rand1 = random.randint(0,self.number_of_genes-2)
        rand2 = random.randint(rand1+1,self.number_of_genes-1)
        for i in range(self.number_of_genes):
            if(i<rand1 or i>rand2):
                self.chromosomes[c1][i] = self.chromosomes[p1][i]
            else:
                index = rand2-(i-rand1)
                self.chromosomes[c1][i] = self.chromosomes[p1][index]

In [20]:
def read_from_excel(filename, n):
    firefighter_df = pd.read_excel("data/" + filename + "_firefighter_route.xlsx")
    fire_df = pd.read_excel("data/" + filename + "_fire_route.xlsx") 
    burning_time_df = pd.read_excel("data/" + filename + "_burning_time.xlsx") 
    processing_time_df = pd.read_excel("data/" + filename + "_processing_time.xlsx") 
    A_p = [[-1 for i in range(n+1)] for j in range(n+1)]
    A_f = [[0 for i in range(n+1)] for j in range(n+1)]
    processing_time = {}
    for i in range(len(fire_df.index)):
        A_f[fire_df.iloc[i]['i']][fire_df.iloc[i]['j']] = fire_df.iloc[i]['travel time']
    print(A_f)
    for i in range(1, n+1):
        for j in range(1, n+1):
            if A_f[i][j] != 0:
                A_f[i][j] += burning_time_df.iloc[i-1]['burning time']
    for i in range(len(firefighter_df.index)):
        A_p[firefighter_df.iloc[i]['j']][firefighter_df.iloc[i]['i']] = firefighter_df.iloc[i]['travel time']
    for i in range(n+1):
        A_p[0][i] = 0
    for i in range(n):
        processing_time[i+1] = processing_time_df.iloc[i]['processing time']
    processing_time[0] = 10000
    data = {'A_p': A_p, 'A_f': A_f, 'processing_time': processing_time, 'n': n}
    return data    

In [26]:
data = read_from_excel("G6", 6)
A_p = np.array(data['A_p'])
A_f = np.array(data['A_f'])
print(np.array(data['A_p']))
print(np.array(data['A_f']))
print(data['processing_time'])

[[0, 0, 0, 0, 0, 0, 0], [0, 0, 5, 2, 3, 0, 0], [0, 5, 0, 2, 0, 0, 0], [0, 2, 2, 0, 2, 0, 0], [0, 3, 0, 2, 0, 4, 0], [0, 0, 0, 0, 4, 0, 0], [0, 0, 0, 0, 0, 0, 0]]
[[ 0  0  0  0  0  0  0]
 [-1 -1  3  1  1 -1 -1]
 [-1  3 -1  1 -1 -1 -1]
 [-1  1  1 -1  1 -1  1]
 [-1  1 -1  1 -1  2 -1]
 [-1 -1 -1 -1  2 -1  1]
 [-1 -1 -1  1 -1  1 -1]]
[[0 0 0 0 0 0 0]
 [0 0 7 4 5 0 0]
 [0 7 0 4 0 0 0]
 [0 4 4 0 4 0 0]
 [0 5 0 4 0 6 0]
 [0 0 0 0 6 0 0]
 [0 0 0 0 0 0 0]]
{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 0, 0: 10000}


In [22]:
class FFProblem:
    def __init__(self,data, N_D, N_F, T):
        self.M = 1000
        self.A_p = np.array(data['A_p'])
        self.A_f = np.array(data['A_f']) 
        self.n = data['n']
        self.N_D = N_D
        self.N_F = N_F
        self.processing_time = data['processing_time']
        self.T = T
        self.A_f_reciprocal = np.array([[0.0 for i in range(self.n+1)]for j in range(self.n+1)])
        for i in range(self.n+1):
            for j in range(self.n+1):
                if A_f[i][j] != 0:
                    self.A_f_reciprocal[i][j] = 1/A_f[i][j]
                    
    def compute_objective_value(self,po):
        protect_order = list(po)
        t = 0
        # curently only deal with single firefighter #
        x = [[0 for i in range(self.n+1)]]
        u = []
        for i in range(self.T): 
            u.append([1 for i in range(self.n+1)])
#             u.append(np.array([1 for i in range(self.n)]))
        u = np.array(u)
        for i in N_D:
            for j in range(self.T):
                u[j][i] = 0
            protect_order.insert(0, i)
        for i in range(self.T):
            u[i][0] = 0
        for i in N_F: x[0][i] = 1

#         print(x[0], u)
        
        for i in range(self.n):
#             print('u:', u)
#             print(self.get_breakout_time(x[0], u))
#             st, dt = protect_order[i]-1, protect_order[i+1]-1
            st, dt = protect_order[i], protect_order[i+1]
            t += self.processing_time[st]
            if t >= self.T: break
            arr = self.move_to_next_node(st, dt, t, self.get_breakout_time(x[0], u)) #st, dt -> 0-based
            if arr == self.M:
                return self.M
                0
            else:
                t += arr
                if t >= self.T: 
                    break
                for j in range(t, self.T): u[j, dt] = 0
         
        process = np.zeros((self.n+1, self.n+1))
        for i in range(self.T):
            process += np.multiply(self.A_f_reciprocal, x[-1])
            x.append([min(math.floor(np.max(process[k, :]))+x[-1][k], 1)*u[i][k] for k in range(self.n+1)])
        x = np.array(x)
#         print(x)
        return np.sum(x[-1].dot(np.ones((self.n+1, 1))))
        
    def get_breakout_time(self, x_original, u):
        x = [x_original]
        process = np.zeros((self.n+1, self.n+1))
        for i in range(self.T):
            process += np.multiply(self.A_f_reciprocal, x[-1])
            x.append([min(math.floor(np.max(process[k, :]))+x[-1][k], 1)*u[i][k] for k in range(self.n+1)])
        result = [self.M] * (self.n+1)
        x = np.array(x)
        for i in range(self.n):
            if 1 in list(x[:, i]):
                result[i] = list(x[:,i]).index(1)
        return result
    
    def move_to_next_node(self, st, dt, t, breakout_time): #st, dt are 1-based
        unvisit = [i for i in range(self.n+1)]
        cost = [self.M for i in range(self.n+1)]
        cost[st] = 0
        path = [-1] * (self.n+1)
        cur = st
        while cur != dt:
            for i, val in enumerate(list(A_p[cur, :])):
#                 if cost[cur] + val < cost[i] and val > 0:
                if cost[cur] + val < cost[i] and t + cost[cur] + val <= breakout_time[i] and val >= 0:
                    path[i] = cur
                    cost[i] = cost[cur] + val
            unvisit.remove(cur)
#             cur = cost.index(min([cost[j] for j in unvisit]))
            if len(unvisit) == 0: break
            temp = []
            for idx in unvisit:
                temp.append((cost[idx], idx))
            cur = min(temp)[1]
        return cost[dt]
        

In [23]:
N_D = [6] # 1 base
N_F = [1] # 1 base
n = 6
data = read_from_excel("G6", n)
T = 5
ff = FFProblem(data, N_D, N_F, T)
protect_order = [5, 4, 3, 2, 1, 0]
ff.compute_objective_value(protect_order)



[[0, 0, 0, 0, 0, 0, 0], [0, 0, 5, 2, 3, 0, 0], [0, 5, 0, 2, 0, 0, 0], [0, 2, 2, 0, 2, 0, 0], [0, 3, 0, 2, 0, 4, 0], [0, 0, 0, 0, 4, 0, 0], [0, 0, 0, 0, 0, 0, 0]]


3.0

In [24]:
N_D = [6] # 1 base
N_F = [1] # 1 base
n = 6
data = read_from_excel("G6", n)
T = 15
ff = FFProblem(data, N_D, N_F, T)
protect_order = [2, 1, 4, 3, 0]
ff.compute_objective_value(protect_order)



[[0, 0, 0, 0, 0, 0, 0], [0, 0, 5, 2, 3, 0, 0], [0, 5, 0, 2, 0, 0, 0], [0, 2, 2, 0, 2, 0, 0], [0, 3, 0, 2, 0, 4, 0], [0, 0, 0, 0, 4, 0, 0], [0, 0, 0, 0, 0, 0, 0]]


1000

In [25]:
ff = FFProblem(data, N_D, N_F, T)

pop_size = 50
# pop_size = 20
selection_type = SelectionType.Deterministic
crossover_type = CrossoverType.PartialMappedCrossover
crossover_rate = 0.2
mutation_type = MutationType.Inversion
mutation_rate = 0.1
solver = GeneticAlgorithm(pop_size,ff.n ,selection_type,
                          crossover_type,crossover_rate,
                          mutation_type,mutation_rate,
                          ff.compute_objective_value)
solver.initialize()

# print(f"{solver.best_chromosome}")

for i in range(100):
    solver.perform_crossover_operation()
    solver.perform_mutation_operation()
    solver.evaluate_fitness()
    solver.update_best_solution()
    solver.perform_selection()
    if(i %10 ==0):
        print(F"iteration {i} :")
        print(f"{solver.best_chromosome}: {ff.compute_objective_value(solver.best_chromosome)}")

iteration 0 :
[2 1 0 4 3 5]: 1000
iteration 10 :
[2 1 0 4 3 5]: 1000
iteration 20 :
[2 1 0 4 3 5]: 1000
iteration 30 :
[2 1 0 4 3 5]: 1000
iteration 40 :
[2 1 0 4 3 5]: 1000
iteration 50 :
[3 4 2 5 1 0]: 1.0
iteration 60 :
[3 4 2 5 1 0]: 1.0
iteration 70 :
[3 4 2 5 1 0]: 1.0
iteration 80 :
[3 4 2 5 1 0]: 1.0
iteration 90 :
[3 4 2 5 1 0]: 1.0
